In [19]:
import numpy as np
import pandas as pd

import pickle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns

import sys
import os


In [20]:
notebook_path = os.path.abspath(".")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..', 'src')))

from limpieza import limpia
import encoding_func

In [21]:
df = limpia.tratamiento_nans("../data/coches_consolidado_limpio.csv")
df.head(3)

,modelo_titulo,provincia,kilometraje,combustible,distintivo_ambiental,garantia,cambio_automatico,carroceria,plazas,potencia,...,referencia,url,ruta_imagen,marca_sola,mes_matricula,anio_matricula,nombre_vendedor,peninsula_y_baleares,comunidad,potencia_segmentado
0,XC40 D4 R-Design Premium Edition AWD Aut.,Madrid,115000,Diésel,C,12.0,True,Todo Terreno,5.0,190.0,...,15483995,https://www.autocasion.com/coches-segunda-mano...,E:\hackABoss\pfb-coches\src\extraction/../../i...,VOLVO,1,2019,INDIVIDUAL AUTO SL\r,True,Comunidad de Madrid,150-200
1,Clase GLC Todoterreno Automático de 5 Puertas,Málaga,15000,Híbrido Enchufable,0 EMISIONES,12.0,True,"4x4, SUV o pickup",5.0,330.0,...,15509915,https://www.autocasion.com/coches-segunda-mano...,E:\hackABoss\pfb-coches\src\extraction/../../i...,MERCEDES-BENZ,5,2023,AUTOS DANIEL\r,True,Andalucía,300-450
2,C5 X 1.6 Puretech Shine EAT8 180,Burgos,36777,Gasolina,C,16.0,True,Berlina,5.0,180.0,...,14540815,https://www.autocasion.com/coches-segunda-mano...,E:\hackABoss\pfb-coches\src\extraction/../../i...,CITROEN,6,2022,"MOTOR ARANDA, VO\r",True,Castilla y León,150-200


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14523 entries, 0 to 14522
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   modelo_titulo         14523 non-null  object 
 1   provincia             14523 non-null  object 
 2   kilometraje           14523 non-null  int64  
 3   combustible           14523 non-null  object 
 4   distintivo_ambiental  14523 non-null  object 
 5   garantia              14523 non-null  float64
 6   cambio_automatico     14523 non-null  bool   
 7   carroceria            14523 non-null  object 
 8   plazas                14523 non-null  float64
 9   potencia              14523 non-null  float64
 10  puertas               14523 non-null  int64  
 11  color                 14523 non-null  object 
 12  precio                14520 non-null  float64
 13  vendedor_profesional  14523 non-null  bool   
 14  consumo_medio         14523 non-null  float64
 15  certificado        

In [23]:
df_modelo = df.drop(columns=["modelo_titulo", "referencia", "url", "ruta_imagen", "fecha_extraccion", "nombre_vendedor", "garantia"])


In [24]:
#Elimino filas con NaN en precio
df_modelo = df_modelo.dropna(subset=["precio"])
df_modelo.isna().sum()


provincia               0
kilometraje             0
combustible             0
distintivo_ambiental    0
cambio_automatico       0
carroceria              0
plazas                  0
potencia                0
puertas                 0
color                   0
precio                  0
vendedor_profesional    0
consumo_medio           0
certificado             0
marca_sola              0
mes_matricula           0
anio_matricula          0
peninsula_y_baleares    0
comunidad               0
potencia_segmentado     0
dtype: int64

In [25]:
encoding_func.generar_pickle_label_encoding(df_modelo,["combustible", "distintivo_ambiental", "carroceria", "color"])
encoding_func.generar_pickle_onehot_encoding(df_modelo, ["cambio_automatico", "vendedor_profesional", "certificado", "peninsula_y_baleares"])

Pickle LE combustible_encoder.pickle generado.
Pickle LE distintivo_ambiental_encoder.pickle generado.
Pickle LE carroceria_encoder.pickle generado.
Pickle LE color_encoder.pickle generado.
Pickle OHE cambio_automatico_encoder.pickle generado.
Pickle OHE vendedor_profesional_encoder.pickle generado.
Pickle OHE certificado_encoder.pickle generado.
Pickle OHE peninsula_y_baleares_encoder.pickle generado.


In [26]:
with open(f"../bin/cambio_automatico_encoder.pickle", "rb") as file:
    cambio_automatico_encoder = pickle.load(file)
    
with open(f"../bin/carroceria_encoder.pickle", "rb") as file:
    carroceria_encoder = pickle.load(file)

with open(f"../bin/certificado_encoder.pickle", "rb") as file:
    certificado_encoder = pickle.load(file)

with open(f"../bin/color_encoder.pickle", "rb") as file:
    color_encoder = pickle.load(file)

with open(f"../bin/combustible_encoder.pickle", "rb") as file:
    combustible_encoder = pickle.load(file)

with open(f"../bin/distintivo_ambiental_encoder.pickle", "rb") as file:
    distintivo_ambiental_encoder = pickle.load(file)

with open(f"../bin/vendedor_profesional_encoder.pickle", "rb") as file:
    vendedor_profesional_encoder = pickle.load(file)
    
with open(f"../bin/peninsula_y_baleares_encoder.pickle", "rb") as file:
    peninsula_y_baleares_encoder = pickle.load(file)


In [27]:
#Aplico la funcion log1p para que no de problemas con los valores cero
df_modelo["precio"] = np.log1p(df_modelo["precio"])
df_modelo["kilometraje"] = np.log1p(df_modelo["kilometraje"])

In [28]:
df_modelo["combustible"] = combustible_encoder.transform(df_modelo["combustible"].astype(str))
df_modelo["carroceria"] = carroceria_encoder.transform(df_modelo["carroceria"].astype(str))
df_modelo["color"] = color_encoder.transform(df_modelo["color"].astype(str))
df_modelo["distintivo_ambiental"] = distintivo_ambiental_encoder.transform(df_modelo["distintivo_ambiental"].astype(str))


In [29]:
df_modelo["cambio_automatico"] = cambio_automatico_encoder.transform(df_modelo[["cambio_automatico"]])
df_modelo["certificado"] = certificado_encoder.transform(df_modelo[["certificado"]])
df_modelo["peninsula_y_baleares"] = peninsula_y_baleares_encoder.transform(df_modelo[["peninsula_y_baleares"]])
df_modelo["vendedor_profesional"] = vendedor_profesional_encoder.transform(df_modelo[["vendedor_profesional"]])


In [30]:
TARGET = "precio"
X_train, X_test, y_train, y_test = encoding_func.dividir_dataframe(df_modelo, TARGET)

encoding_func.generar_pickle_target_encoding(X_train, y_train, ["provincia", "marca_sola", "potencia_segmentado", "comunidad"], TARGET)

Pickle TE provincia_precio_encoder.pickle generado.
Pickle TE marca_sola_precio_encoder.pickle generado.
Pickle TE potencia_segmentado_precio_encoder.pickle generado.
Pickle TE comunidad_precio_encoder.pickle generado.


In [31]:
with open(f"../bin/marca_sola_precio_encoder.pickle", "rb") as file:
    marca_sola_precio_encoder = pickle.load(file)

with open(f"../bin/potencia_segmentado_precio_encoder.pickle", "rb") as file:
    potencia_segmentado_precio_encoder = pickle.load(file)

with open(f"../bin/provincia_precio_encoder.pickle", "rb") as file:
    provincia_precio_encoder = pickle.load(file)

with open(f"../bin/comunidad_precio_encoder.pickle", "rb") as file:
    comunidad_precio_encoder = pickle.load(file)


In [32]:
X_train["provincia"] = provincia_precio_encoder.transform(X_train["provincia"])
X_train["marca_sola"] = marca_sola_precio_encoder.transform(X_train["marca_sola"])
X_train["potencia_segmentado"] = potencia_segmentado_precio_encoder.transform(X_train["potencia_segmentado"])
X_train["comunidad"] = comunidad_precio_encoder.transform(X_train["comunidad"])


X_test["provincia"] = provincia_precio_encoder.transform(X_test["provincia"])
X_test["marca_sola"] = marca_sola_precio_encoder.transform(X_test["marca_sola"])
X_test["potencia_segmentado"] = potencia_segmentado_precio_encoder.transform(X_test["potencia_segmentado"])
X_test["comunidad"] = comunidad_precio_encoder.transform(X_test["comunidad"])

In [37]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11616 entries, 5516 to 7272
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   provincia             11616 non-null  float64
 1   kilometraje           11616 non-null  float64
 2   combustible           11616 non-null  int64  
 3   distintivo_ambiental  11616 non-null  int64  
 4   cambio_automatico     11616 non-null  float64
 5   carroceria            11616 non-null  int64  
 6   plazas                11616 non-null  float64
 7   potencia              11616 non-null  float64
 8   puertas               11616 non-null  int64  
 9   color                 11616 non-null  int64  
 10  vendedor_profesional  11616 non-null  float64
 11  consumo_medio         11616 non-null  float64
 12  certificado           11616 non-null  float64
 13  marca_sola            11616 non-null  float64
 14  mes_matricula         11616 non-null  int64  
 15  anio_matricula        

In [36]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2904 entries, 11296 to 562
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   provincia             2904 non-null   float64
 1   kilometraje           2904 non-null   float64
 2   combustible           2904 non-null   int64  
 3   distintivo_ambiental  2904 non-null   int64  
 4   cambio_automatico     2904 non-null   float64
 5   carroceria            2904 non-null   int64  
 6   plazas                2904 non-null   float64
 7   potencia              2904 non-null   float64
 8   puertas               2904 non-null   int64  
 9   color                 2904 non-null   int64  
 10  vendedor_profesional  2904 non-null   float64
 11  consumo_medio         2904 non-null   float64
 12  certificado           2904 non-null   float64
 13  marca_sola            2904 non-null   float64
 14  mes_matricula         2904 non-null   int64  
 15  anio_matricula        2

In [34]:
X_train.head(3)

,provincia,kilometraje,combustible,distintivo_ambiental,cambio_automatico,carroceria,plazas,potencia,puertas,color,vendedor_profesional,consumo_medio,certificado,marca_sola,mes_matricula,anio_matricula,peninsula_y_baleares,comunidad,potencia_segmentado
5516,9.955145,11.968735,0,2,1.0,4,5.0,177.0,5,1,1.0,4.200000,0.0,10.464717,7,2015,1.0,10.033486,10.189896
3893,9.917819,11.259683,2,2,0.0,7,5.0,130.0,5,0,1.0,5.100000,0.0,9.764679,4,2019,1.0,9.917819,9.836951
640,10.044698,10.860824,2,2,0.0,7,5.0,150.0,5,1,1.0,5.288752,0.0,9.980030,3,2023,1.0,10.097616,10.189896


In [35]:
scaler = MinMaxScaler()
scaler.set_output(transform="pandas")

X_train_escalado = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_escalado = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

with open(f"../bin/min_max_scaler.pickle", "wb") as file:
    pickle.dump(scaler, file)